# Database

In [ ]:
import pandas as pd
import numpy as np
import os
from utils.date_generation import get_date_string
from sklearn.preprocessing import MultiLabelBinarizer

%load_ext autoreload
%autoreload 2

### Create a data

In [ ]:
TEACHERS = ['Дикун И.А. professor','Козлова Л.П. professor','Королев В.В. professor',
            'Кузнецов В.Е. professor','Якупов О.Э. assistant','Буканин В.А. professor',
            'Демидович О.В. professor','Овдиенко Е.Н. assistant','Трусов А.А. assistant', 'Трусов А.О. professor',
            'Леута А.А. professor','Мирошников А.Н. professor','Амбросовская Е.Б. professor',
            'Лукичев А.Н. professor','Михайлов Д.П. assistant','Копычев М.М. professor', 'Игнатович Ю.В. assistant',
            'Скороходов Д.А. professor','Вейнмейстер А.В. professor','Филатова Е.С. professor',
            'Богданова С.М. assistant','Федоркова А.О. assistant']


In [ ]:
SUBJECTS = ['Привод пр.','Привод лек.','Привод лаб.',
            'ТССУ лек.','ТССУ пр.','ТССУ кр.','ТССУ лаб.',
            'БЖД лек.','БЖД пр.','БЖД лаб.','АПМСЭЭС лек.',
            'АПМСЭЭС лаб.','МССУ (ДВС6) лек.','МССУ (ДВС6) лаб.',
            'МССУ (ДВС6) пр.','ТОУ лек.','ТОУ лаб.',
            'СЭЭС лек.','СЭЭС пр.','СЭЭС лаб.',
            'МПТвМиРТ лек.','МПТвМиРТ лаб.','МПТвМиРТ пр.',
            'ФиРИСУК (ДВС8) лек.','ФиРИСУК (ДВС8) пр.',
            'ФиРИСУК (ДВС8) лаб.','СУЭЭУК пр.','СУЭЭУК лаб.','СУЭЭУК лек.',
            'ПЛКиПС лек.','ПЛКиПС лаб.','ПЛКиПС пр.',
            'ТЕХНИЧЕСКОЕ ЗРЕНИЕ','ТЕХНИЧЕСКОЕ ЗРЕНИЕ лаб.','ТЕХНИЧЕСКОЕ ЗРЕНИЕ пр.']

In [ ]:
#Добавить инфу про нагрузку каждого преподавателя. Т.е. посчитать сколько часов из расписания у каждого преподавателя, нужны цифры за 1 неделю, за 2 недели и за весь семестр
TEACHERS = ['Дикун И.А. professor','Козлова Л.П. professor','Королев В.В. professor',
            'Кузнецов В.Е. professor','Якупов О.Э. assistant','Буканин В.А. professor',
            'Демидович О.В. professor','Овдиенко Е.Н. assistant','Трусов А.А. assistant', 'Трусов А.О. professor',
            'Леута А.А. professor','Мирошников А.Н. professor','Амбросовская Е.Б. professor',
            'Лукичев А.Н. professor','Михайлов Д.П. assistant','Копычев М.М. professor', 'Игнатович Ю.В. assistant',
            'Скороходов Д.А. professor','Вейнмейстер А.В. professor','Филатова Е.С. professor',
            'Богданова С.М. assistant','Федоркова А.О. assistant']
#18 недель (вместе с зач. нед.), кол-во пар для каждого преподователя за неделю.
TEACHERS_HOURS_ONE_WEEK = np.array([2, 5, 4,
                                    5, 4, 2,
                                    1, 1, 2, 2,
                                    2, 2, 5,
                                    2, 2, 1.5, 1.5,
                                    2, 2, 1.5,
                                    1.5, 2
                                    ])
#кол-во пар для каждого преподователя за две недели.
TEACHERS_HOURS_TWO_WEEKS = TEACHERS_HOURS_ONE_WEEK[:]*2
#кол-во пар для каждого преподователя за семестр.
TEACHERS_HOURS_ALL = TEACHERS_HOURS_ONE_WEEK[:]*18
print(len(TEACHERS))
print(len(TEACHERS_HOURS_ONE_WEEK))


In [ ]:
TEACHERS_HOURS = {
    'TEACHERS' : ['Дикун И.А. professor','Козлова Л.П. professor','Королев В.В. professor',
            'Кузнецов В.Е. professor','Якупов О.Э. assistant','Буканин В.А. professor',
            'Демидович О.В. professor','Овдиенко Е.Н. assistant','Трусов А.А. assistant', 'Трусов А.О. professor',
            'Леута А.А. professor','Мирошников А.Н. professor','Амбросовская Е.Б. professor',
            'Лукичев А.Н. professor','Михайлов Д.П. assistant','Копычев М.М. professor', 'Игнатович Ю.В. assistant',
            'Скороходов Д.А. professor','Вейнмейстер А.В. professor','Филатова Е.С. professor',
            'Богданова С.М. assistant','Федоркова А.О. assistant'],
    'TEACHERS_HOURS_ONE_WEEK' : np.array([2, 5, 4,
                                    5, 4, 2,
                                    1, 1, 2, 2,
                                    2, 2, 5,
                                    2, 2, 1.5, 1.5,
                                    2, 2, 1.5,
                                    1.5, 2
                                    ]),
    'TEACHERS_HOURS_TWO_WEEKS' : TEACHERS_HOURS_ONE_WEEK[:]*2,
    'TEACHERS_HOURS_ALL' : TEACHERS_HOURS_ONE_WEEK[:]*18        
}

In [ ]:
#Добавить инфу про кол-во часов по каждому предмету. Нужны часы за 1 неделю, за 2 недели и за весь семестр

#кол-во пар в неделю для 4 групп.
SUBJECTS_HOURS_ONE_WEEK = np.array([4, 1, 4,
                          1, 2, 2, 4,
                          1, 4, 4, 1,
                          1, 1, 0.5,
                          0.5, 1, 4,
                          1, 0.5, 0.5,
                          1, 1, 1,
                          1, 0.5,
                          0.5, 0.5, 0.5, 1,
                          1, 1.5, 1.5,
                          1, 1, 1])
#кол-во пар за две недели для 4 групп.
SUBJECTS_HOURS_TWO_WEEKS = SUBJECTS_HOURS_ONE_WEEK[:36]*2
#18 недель (вместе с зач. нед.), кол-во пар для 1 группы.
SUBJECTS_HOURS_ALL = np.array([18, 18, 16,
                  18, 9, 9, 18,
                  18, 18, 18, 18,
                  18, 18, 9, 
                  9, 18, 18,
                  18, 9, 9,
                  18, 9, 8,
                  18, 9, 
                  9, 9, 9, 18,
                  18, 9, 9,
                  9, 9, 9])

In [ ]:
SUBJECTS_HOURS = {
    'SUBJECTS' : ['Привод пр.','Привод лек.','Привод лаб.',
            'ТССУ лек.','ТССУ пр.','ТССУ кр.','ТССУ лаб.',
            'БЖД лек.','БЖД пр.','БЖД лаб.','АПМСЭЭС лек.',
            'АПМСЭЭС лаб.','МССУ (ДВС6) лек.','МССУ (ДВС6) лаб.',
            'МССУ (ДВС6) пр.','ТОУ лек.','ТОУ лаб.',
            'СЭЭС лек.','СЭЭС пр.','СЭЭС лаб.',
            'МПТвМиРТ лек.','МПТвМиРТ лаб.','МПТвМиРТ пр.',
            'ФиРИСУК (ДВС8) лек.','ФиРИСУК (ДВС8) пр.',
            'ФиРИСУК (ДВС8) лаб.','СУЭЭУК пр.','СУЭЭУК лаб.','СУЭЭУК лек.',
            'ПЛКиПС лек.','ПЛКиПС лаб.','ПЛКиПС пр.',
            'ТЕХНИЧЕСКОЕ ЗРЕНИЕ','ТЕХНИЧЕСКОЕ ЗРЕНИЕ лаб.','ТЕХНИЧЕСКОЕ ЗРЕНИЕ пр.'],
    'SUBJECTS_HOURS_ONE_WEEK' : np.array([4, 1, 4,
                          1, 2, 2, 4,
                          1, 4, 4, 1,
                          1, 1, 0.5,
                          0.5, 1, 4,
                          1, 0.5, 0.5,
                          1, 1, 1,
                          1, 0.5,
                          0.5, 0.5, 0.5, 1,
                          1, 1.5, 1.5,
                          1, 1, 1]),
    'SUBJECTS_HOURS_TWO_WEEKS' : SUBJECTS_HOURS_ONE_WEEK[:36]*2,
    'SUBJECTS_HOURS_ALL' : np.array([18, 18, 16,
                  18, 9, 9, 18,
                  18, 18, 18, 18,
                  18, 18, 9, 
                  9, 18, 18,
                  18, 9, 9,
                  18, 9, 8,
                  18, 9, 
                  9, 9, 9, 18,
                  18, 9, 9,
                  9, 9, 9])           
}

In [ ]:
GROUPS = ['9491,9492,9493,9494','9491,9492,9493,9494','9491,9492,9493,9494',
            '9491,9492,9493,9494','9491,9492,9493,9494','9491,9492,9493,9494',
            '9491,9492,9493,9494','9491,9492,9493,9494','9491,9492,9493,9494',
            '9491,9492,9493,9494',
            '9493','9493','9494','9494','9494',
            '9491,9492,9493,9494','9491,9492,9493,9494',
            '9493','9493','9493',
            '9491,9492','9491,9492','9491,9492',
            '9494','9494','9494',
            '9494','9494','9494',
            '9494','9494','9494',
            '9491,9492','9491,9492','9491,9492'
]

In [ ]:
ROOM = ['4302','1229','D103','5230','2403','2404','8103-1',
            '5143','5141','5135','5134','5221','8101','8112','8217','2322','8102',
            '1158','8204','8203-2','8203-1']

In [ ]:
SUBJECT_TYPE = ['пр.','лек.','лаб.','лек.','пр.','пр.','лаб.','лек.',
                'пр.','лаб.','пр.','лек.','лаб.','лаб.','лаб.','лек.','лаб.','лек.','лаб.',
                'пр.','лаб.']


In [ ]:
ROOM_SUBJECT_TYPE = {
    'room': ['4302','1229','D103','5230','2403','2404','8103-1',
                '5143','5141','5135','5134','5221','8101','8112','8217','2322','8102',
                '1158','8204','8203-2','8203-1'],
    'subject_type':['пр.','лек.','лаб.','лек.','пр.','пр.','лаб.','лек.',
                    'пр.','лаб.','пр.','лек.','лаб.','лаб.','лаб.','лек.','лаб.','лек.','лаб.',
                    'пр.','лаб.']
}


In [ ]:
SUBJECT_GROUPS = {
    'subjects': ['Привод пр.','Привод лек.','Привод лаб.',
                'ТССУ лек.','ТССУ пр.','ТССУ кр.','ТССУ лаб.',
                'БЖД лек.','БЖД пр.','БЖД лаб.','АПМСЭЭС лек.',
                'АПМСЭЭС лаб.','МССУ (ДВС6) лек.','МССУ (ДВС6) лаб.',
                'МССУ (ДВС6) пр.','ТОУ лек.','ТОУ лаб.',
                'СЭЭС лек.','СЭЭС пр.','СЭЭС лаб.',
                'МПТвМиРТ лек.','МПТвМиРТ лаб.','МПТвМиРТ пр.',
                'ФиРИСУК (ДВС8) лек.','ФиРИСУК (ДВС8) пр.',
                'ФиРИСУК (ДВС8) лаб.','СУЭЭУК пр.','СУЭЭУК лаб.','СУЭЭУК лек.',
                'ПЛКиПС лек.','ПЛКиПС лаб.','ПЛКиПС пр.',
                'ТЕХНИЧЕСКОЕ ЗРЕНИЕ','ТЕХНИЧЕСКОЕ ЗРЕНИЕ лаб.','ТЕХНИЧЕСКОЕ ЗРЕНИЕ пр.'],
    'groups': [[9491,9492,9493,9494],[9491,9492,9493,9494],[9491,9492,9493,9494],
                [9491,9492,9493,9494],[9491,9492,9493,9494],[9491,9492,9493,9494],[9491,9492,9493,9494],
                [9491,9492,9493,9494],[9491,9492,9493,9494],[9491,9492,9493,9494],[9493],
                [9493],[9494],[9494],
                [9494],[9491,9492,9493,9494],[9491,9492,9493,9494],
                [9493],[9493],[9493],
                [9491,9492],[9491,9492],[9491,9492],
                [9494],[9494],
                [9494],[9494],[9494],[9494],
                [9491,9492,9494],[9491,9492,9494],[9491,9492,9494],
                [9491,9492],[9491,9492],[9491,9492]]
}


In [ ]:
STUDY_TIME = ['8:00-9:30','9:50-11:20','11:40-13:10','13:40-15:10','15:30-17:00','17:20-18:50']

### Unique values

In [ ]:
UNQ_GROUPS = ['9491', '9492', '9493', '9494']
UNQ_SUBJECT_TYPE = ["пр.", "лек.", "лаб."]

In [ ]:
LAB_ROOM = {
    'lab': ['Привод','ТССУ','БЖД','АПМСЭЭС','МССУ (ДВС6)','ТОУ','СЭЭС','МПТвМиРТ','ФиРИСУК (ДВС8)','СУЭЭУК','ПЛКиПС','ТЕХНИЧЕСКОЕ ЗРЕНИЕ'],
    'room':['D103','8103-1','5135','5221','8112','8217','8102','8204','8112','8102','8103-1','8203-1']
}

In [ ]:
feature_arry = pd.DataFrame(SUBJECT_GROUPS)

### One-Hot-Encoding

In [ ]:
mlb = MultiLabelBinarizer(sparse_output=True)
dsubjects_groups = feature_arry.join(pd.DataFrame.sparse.from_spmatrix(mlb.fit_transform(feature_arry.pop('groups')), index = feature_arry.index, columns=mlb.classes_))

### Create a dataframe

In [ ]:
droom_subject_type = pd.DataFrame(ROOM_SUBJECT_TYPE)

In [ ]:
dteachers = pd.DataFrame(data=TEACHERS,columns=['teachers'])

In [ ]:
dlab_room = pd.DataFrame(data=LAB_ROOM)

In [ ]:
dsubjects_hours = pd.DataFrame(SUBJECTS_HOURS)

In [ ]:
dteachers_hours = pd.DataFrame(TEACHERS_HOURS)

### Create a path

In [ ]:
#CURRENT_DPATH = os.path.abspath("01_data_generation.ipynb")
#print(CURRENT_DPATH)
CURRENT_DPATH = os.path.dirname(os.path.abspath("01_data_generation.ipynb"))
print(CURRENT_DPATH)
PROJECT_ROOT = os.path.abspath(os.path.join(CURRENT_DPATH, os.pardir))
print(PROJECT_ROOT)
DATA_DPATH = os.path.join(PROJECT_ROOT, "data")
print(DATA_DPATH)
save_dpath = os.path.join(DATA_DPATH, get_date_string())
print(save_dpath)

### Write to disk

In [ ]:
save_dpath = os.path.join(DATA_DPATH, get_date_string())
os.makedirs(save_dpath, exist_ok=True) 

dsubjects_groups.to_csv(os.path.join(save_dpath, "Subjects+Groups.csv"))
dsubjects_hours.to_csv(os.path.join(save_dpath, "Subjects+Hours.csv"))
dteachers_hours.to_csv(os.path.join(save_dpath, "Teachers+Hours.csv"))
droom_subject_type.to_csv(os.path.join(save_dpath, "Room+Subject_Type.csv"))
dlab_room.to_csv(os.path.join(save_dpath, "Lab+Room.csv"))
